In [1]:
import gym
import gym_maze

MAZE = 'MazeXYZ-v0' # SET PROPER ENV    # if state as strings list - use observation wrapper

maze = gym.make(MAZE)

In [2]:
from ray.rllib.agents.dqn import DQNTrainer
import numpy as np
import os
from ray.tune.registry import register_env


RESULTS_PATH = os.path.join('RESULTS', 'RL_RESULTS')
DQN_RESULTS_PATH = os.path.join(RESULTS_PATH, 'DQN', MAZE)
os.makedirs(DQN_RESULTS_PATH, exist_ok=True)
def maze_env_creator(env_config):
    maze.reset()
    return maze  # return an env instance

register_env("maze_env", maze_env_creator)

RUNS = 30
RUNS_START = 1

for run in range(RUNS_START, RUNS_START + RUNS):
# Create an RLlib Trainer instance to learn how to act in the above
# environment.
    trainer = DQNTrainer(
        config={
            # Env class to use (here: our gym.Env sub-class from above).
            "env": "maze_env",
            # Config dict to be passed to our custom env's constructor.
            "env_config": {
            },
            # Parallelize environment rollouts.
            "num_workers": 0,
            "n_step": 10,
            "noisy": True,
            "num_atoms": 4,
            "v_min": 0.0, 
            "v_max": 1000.0,
            "batch_mode": "complete_episodes",
            "exploration_config": {
            # The Exploration class to use.
                "type": "EpsilonGreedy",
                # Config for the Exploration class' constructor:
                "initial_epsilon": 0.5,
                "final_epsilon": 0.5,
                "epsilon_timesteps": 50000000,  # Timesteps over which to anneal epsilon.
            },
            "evaluation_config": {
                "explore": True,
            },
        })


    total_episodes = 0

    all_episodes_steps = []
    i = 0
    while total_episodes < 5000:
        i += 1
        results = trainer.train()
        iteration_episodes = results['episodes_this_iter']
        total_episodes += iteration_episodes
        print(f'TOTAL EPISODES: {total_episodes}')
        print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}; # episodes_steps={results['hist_stats']['episode_lengths']}")
        all_episodes_steps += results['hist_stats']['episode_lengths']

    np.save(os.path.join(DQN_RESULTS_PATH, f'{run}.npy'), np.array(all_episodes_steps))


2022-06-08 01:28:07,525	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).


TOTAL EPISODES: 21
Iter: 1; avg. reward=95.23809523809524; # episodes_steps=[50, 50, 50, 50, 15, 50, 50, 37, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 43
Iter: 2; avg. reward=116.27906976744185; # episodes_steps=[50, 50, 50, 50, 15, 50, 50, 37, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 20, 50, 50, 29, 50, 50, 50, 50, 50, 50, 22, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 64
Iter: 3; avg. reward=93.75; # episodes_steps=[50, 50, 50, 50, 15, 50, 50, 37, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 20, 50, 50, 29, 50, 50, 50, 50, 50, 50, 22, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 86
Iter: 4; avg. reward=116.27906976744185; # episodes_steps=[50, 50, 50, 50, 15, 50, 50, 37, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 20, 50, 50, 29, 50, 50, 50, 50, 50, 50, 22, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 2, 50, 

2022-06-08 01:40:55,469	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).


TOTAL EPISODES: 5028
Iter: 147; avg. reward=780.0; # episodes_steps=[5, 14, 22, 11, 34, 50, 50, 19, 28, 38, 50, 4, 17, 16, 50, 50, 27, 31, 3, 43, 50, 6, 25, 27, 12, 29, 50, 35, 36, 50, 19, 2, 5, 16, 50, 50, 18, 39, 7, 50, 24, 15, 25, 50, 23, 50, 50, 23, 39, 23, 50, 11, 18, 15, 39, 50, 6, 18, 48, 21, 30, 23, 23, 11, 21, 32, 11, 44, 3, 6, 16, 36, 31, 18, 5, 23, 1, 33, 50, 50, 2, 1, 2, 11, 14, 22, 50, 28, 35, 50, 19, 26, 3, 50, 50, 22, 26, 28, 2, 16]


TOTAL EPISODES: 23
Iter: 1; avg. reward=173.91304347826087; # episodes_steps=[50, 50, 50, 50, 50, 50, 50, 50, 50, 2, 23, 50, 50, 50, 50, 50, 50, 50, 32, 50, 50, 11, 50]
TOTAL EPISODES: 45
Iter: 2; avg. reward=133.33333333333334; # episodes_steps=[50, 50, 50, 50, 50, 50, 50, 50, 50, 2, 23, 50, 50, 50, 50, 50, 50, 50, 32, 50, 50, 11, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 27, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 68
Iter: 3; avg. reward=147.05882352941177; # episodes_steps=[50, 50, 50, 50, 50, 50, 50, 50, 50, 2, 23, 50, 50, 50, 50, 50, 50, 50, 32, 50, 50, 11, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 27, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 14, 50, 50, 50, 50, 50, 50, 50, 50, 16, 50, 50, 47, 50, 50, 50, 50, 50, 50, 50, 17, 50, 50]
TOTAL EPISODES: 91
Iter: 4; avg. reward=164.83516483516485; # episodes_steps=[50, 50, 50, 50, 50, 50, 50, 50, 50, 2, 23, 50, 50, 50, 50, 50, 50, 50, 32, 50, 50, 11, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 27, 50, 50, 50, 50, 50,

...

TOTAL EPISODES: 23
Iter: 1; avg. reward=173.91304347826087; # episodes_steps=[50, 50, 50, 50, 50, 50, 50, 2, 6, 43, 50, 50, 50, 50, 50, 50, 1, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 46
Iter: 2; avg. reward=217.3913043478261; # episodes_steps=[50, 50, 50, 50, 50, 50, 50, 2, 6, 43, 50, 50, 50, 50, 50, 50, 1, 50, 50, 50, 50, 50, 50, 5, 45, 50, 50, 50, 50, 50, 50, 50, 50, 40, 31, 45, 50, 50, 50, 1, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 70
Iter: 3; avg. reward=228.57142857142858; # episodes_steps=[50, 50, 50, 50, 50, 50, 50, 2, 6, 43, 50, 50, 50, 50, 50, 50, 1, 50, 50, 50, 50, 50, 50, 5, 45, 50, 50, 50, 50, 50, 50, 50, 50, 40, 31, 45, 50, 50, 50, 1, 50, 50, 50, 50, 50, 50, 50, 22, 50, 50, 50, 50, 50, 21, 50, 50, 50, 50, 47, 8, 50, 50, 50, 50, 6, 50, 36, 50, 50, 50]
TOTAL EPISODES: 91
Iter: 4; avg. reward=197.8021978021978; # episodes_steps=[50, 50, 50, 50, 50, 50, 50, 2, 6, 43, 50, 50, 50, 50, 50, 50, 1, 50, 50, 50, 50, 50, 50, 5, 45, 50, 50, 50, 50, 50, 50, 50, 50, 40, 31, 45, 50, 50, 5

2022-06-08 05:57:06,330	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).


TOTAL EPISODES: 22
Iter: 1; avg. reward=90.9090909090909; # episodes_steps=[50, 50, 30, 50, 50, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 42
Iter: 2; avg. reward=47.61904761904762; # episodes_steps=[50, 50, 30, 50, 50, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 63
Iter: 3; avg. reward=47.61904761904762; # episodes_steps=[50, 50, 30, 50, 50, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 11, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 85
Iter: 4; avg. reward=58.8235294117647; # episodes_steps=[50, 50, 30, 50, 50, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 11

2022-06-08 06:04:26,720	WARNING rollout_worker.py:498 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).


TOTAL EPISODES: 25
Iter: 1; avg. reward=200.0; # episodes_steps=[50, 50, 28, 5, 50, 4, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 11, 50, 50, 50, 50, 2, 50]
TOTAL EPISODES: 47
Iter: 2; avg. reward=148.93617021276594; # episodes_steps=[50, 50, 28, 5, 50, 4, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 11, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 11, 50, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50]
TOTAL EPISODES: 69
Iter: 3; avg. reward=144.92753623188406; # episodes_steps=[50, 50, 28, 5, 50, 4, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 11, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 11, 50, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 15, 32, 50, 50, 50, 50, 4, 50, 50, 50, 50, 50, 50, 50, 50]
TOTAL EPISODES: 90
Iter: 4; avg. reward=122.22222222222223; # episodes_steps=[50, 50, 28, 5, 50, 4, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 11, 50, 50, 50, 50, 2, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 11, 50, 50, 